In [ ]:
import os
import numpy as np
from glob import glob
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import json
import pandas as pd

- keypoint 는 [17, 3] 의 2차원 배열
- 17는 키포인트의 개수이고 3은 [x, y, visibility] 이다
- visibility {0:안보여서 알수없음, 1:가려짐, 2:보임)

In [ ]:
NUM_KEYPOINT = 17 # 키포인트 개수
NUM_KEYPOINT_CH = 3 # x, y, visibility
NUM_KEYPOINT_CLASS = 3
thresh_keypoint_count = 10 # 데이터 선별시 최소 개수
VISIBILITY_NOT_EXIST = 0
VISIBILITY_OCCLUDED = 1
VISIBILITY_VISIBLE = 2

In [ ]:
max_data_m = 20
channel_label = 4 + 1 + NUM_KEYPOINT * NUM_KEYPOINT_CH #(box:4, cls:1, keypoint:17*3)
channel_label

In [ ]:
folder_COCO = '/media/mvlab/469B5B3C650FBA77/data/COCO/'
folder_COCO_annotation = folder_COCO + 'annotations_trainval2017/annotations/'
folder_img = 'val2017'
#folder_img = 'train2017'

In [ ]:
path_annotation_keypoints_val2017 = folder_COCO_annotation + 'person_keypoints_'+folder_img + '.json'

In [ ]:
os.path.isdir(folder_COCO), os.path.isdir(folder_COCO_annotation)

In [ ]:
os.path.isfile(path_annotation_keypoints_val2017)

In [ ]:
with open(path_annotation_keypoints_val2017) as json_file:
    json_data = json.load(json_file)
    
len(json_data), json_data.keys()

In [ ]:
class_keypoints = json_data['categories'][0]['keypoints']
images = json_data['images']
annotations = json_data['annotations']

len(class_keypoints), len(images), len(annotations)

In [ ]:
class_keypoints

#### AIHub 사람 동작 영상 AI 데이터는 키포인트 16개
- 추가 4 head, neck, chest, hip
- 제거 5 nose, left_eye, right_eye, left_ear, right_ear

##### COCO VS AIHub
- COCO 얼굴 중요(눈, 코, 귀)
- AIHub 동작 중요(목, 가슴, 골반 중심)|

In [ ]:
images[0]

In [ ]:
annotations[0].keys()

In [ ]:
annotations[0]['id']

In [ ]:
annotations[0]['bbox']

In [ ]:
dict_id_image_info = dict()
image_w = []
image_h = []
for image_info in images:
    #print('image_info', image_info)    
    
    image_w.append(image_info['width'])
    image_h.append(image_info['height'])
    
    image_id = image_info['id']    
    dict_id_image_info[image_id] = image_info
        
len(dict_id_image_info)

In [ ]:
np.min(image_w), np.min(image_h), np.max(image_w), np.max(image_h)

In [ ]:
(np.array(image_w) < 200).mean(), (np.array(image_h) < 200).mean()

In [ ]:
plt.title('w, h')
plt.scatter(image_w, image_h, s=2)
plt.xlabel('width')
plt.ylabel('height')

In [ ]:
# get image_filename from image_id
list_id = []
list_category_id = []
for annotation in annotations:
    
    image_id = annotation['image_id']
    category_id = annotation['category_id']    
    
    image_info = dict_id_image_info[image_id]
    if len(list_id) == 0:
        print('category_id', category_id)
        print('annotation', annotation)
        print('image_info', image_info)
        
    file_name = image_info['file_name']
    height = image_info['height']
    width = image_info['width']
    
    file_full_path = folder_COCO + folder_img + os.sep + file_name
    
    if not os.path.isfile(file_full_path):
        print('not exist', file_full_path)
        continue
        
    annotation['file_name'] = file_full_path
    annotation['height'] = height
    annotation['width'] = width
    
    #print(annotation)
    list_id.append(image_id)
    list_category_id.append(category_id)
    #break

In [ ]:
# 전부 사람(1) 카테고리
np.unique(list_category_id)

In [ ]:
#generate dictionary(key:filename, value:annotation list)
#박스와 클래스, 키포인트를 통합
dict_annotation_coco = dict()
for annotation in annotations:
    #print(annotation)
    segmentation = annotation['segmentation']
    bbox = annotation['bbox']
    category_id = annotation['category_id']
    keypoints = annotation['keypoints']    
    file_name = annotation['file_name']
    
    #x0, y0, x0 + w, y0 + h
    x0, y0, w, h = bbox
    b = np.array([x0, y0, x0 + w, y0 + h]).flatten()
    c = np.array(category_id).flatten()
    k = np.array(keypoints).flatten()
    #print(len(b), 'box', bbox, len(c), len(k))
    cbox = np.concatenate((b, c, k)).reshape((1, -1))
    
    if len(dict_annotation_coco) == 0:        
        print('file_name', file_name)
        print('bbox', bbox)
        print('segmentation', len(segmentation), len(segmentation[0]))
        print('keypoints', np.array(keypoints).reshape((-1, 3)))    
        print('cbox', len(cbox), cbox)
    
    if file_name in dict_annotation_coco.keys():
        pre_annotation = dict_annotation_coco[file_name]
        new_bbox = np.concatenate((pre_annotation, cbox), axis=0)        
        dict_annotation_coco[file_name] = new_bbox
    else:
        dict_annotation_coco[file_name] = cbox
    #break

In [ ]:
len(dict_annotation_coco), list(dict_annotation_coco.keys())[0]

In [ ]:
len(list_id), len(set(list_id))

In [ ]:
box_keypoint_label = np.concatenate(list(dict_annotation_coco.values()), 0)
box_keypoint_label.shape

In [ ]:
x0 = box_keypoint_label[:, 0]
y0 = box_keypoint_label[:, 1]
x1 = box_keypoint_label[:, 2]
y1 = box_keypoint_label[:, 3]
box_w = x1-x0
box_h = y1-y0
plt.title('h / w = ' +  str(np.mean(box_h/box_w)))
ax = plt.hist(box_w )
ax = plt.hist(box_h, alpha=0.5)

데이터 선별
- 한사람의 키포인트가 일정 개수 이상 있음

In [ ]:
len(dict_annotation_coco)

In [ ]:
dict_annotation_coco_select = dict()

for key in dict_annotation_coco:
    value = dict_annotation_coco[key]
    #print('key', key)
    #print('value', type(value), value.shape, value)
    
    box = value[:, :4]
    cls = value[:, 4]
    keypoint = value[:, 5:]
    keypoint = keypoint.reshape((-1, NUM_KEYPOINT, NUM_KEYPOINT_CH))
    visibility = keypoint[:, :, 2]
    visibility_count_per_person = np.sum(visibility > VISIBILITY_NOT_EXIST, -1)
    visibility_max_count_per_person = np.max(visibility_count_per_person)
        
    if visibility_max_count_per_person >= thresh_keypoint_count:                
        dict_annotation_coco_select[key] = value

len(dict_annotation_coco), len(dict_annotation_coco_select)
#train2017:(64115, 45900), val2017:(2693, 2012)

In [ ]:
dict_annotation_coco = dict_annotation_coco_select

In [ ]:
def draw_cbox(image, cbox):
    draw = ImageDraw.Draw(image)

    x0, y0, x1, y1, cls = cbox
    draw.text((x0+2, y0), str(int(cls)))
    draw.rectangle((x0, y0, x1, y1), fill=None, outline=(0,255,0), width=2)

In [ ]:
def draw_keypoint(image, keypoints, is_show_class=True):
    draw = ImageDraw.Draw(image)
    
    #print('draw_keypoint', keypoints.shape, keypoints[:,2])
    #[0. 0. 0. 0. 0. 1. 2. 2. 2. 2. 2. 2. 2. 0. 2. 0. 2.]
    for i in range(len(keypoints)):
        point = keypoints[i]
        cls_name = str(i)+':'+ class_keypoints[i] if is_show_class else ''                
        cx = point[0]
        cy = point[1]
        state = point[2]
        circle_radius = 2
        if state > 1:
            draw.ellipse((cx-circle_radius, cy-circle_radius, cx+circle_radius, cy+circle_radius),fill=None, width=1)
            draw.text((cx, cy-10), cls_name)    

In [ ]:
def draw_valid_line(draw, p0, p1, color):
    #point : [x, y, state]
    state0 = p0[2]
    state1 = p1[2]
    if state0 * state1 > 0:
        draw.line((tuple(p0[:2]), tuple(p1[:2])), fill=color)
    

def draw_keypoint_line(image, keypoints):
    draw = ImageDraw.Draw(image)
        
    nose = keypoints[0]
    left_eye = keypoints[1]
    right_eye = keypoints[2]
    left_ear = keypoints[3]
    right_ear = keypoints[4]            
    left_shoulder = keypoints[5]
    right_shoulder = keypoints[6]
    left_elbow = keypoints[7]
    right_elbow = keypoints[8]
    left_wrist = keypoints[9]
    right_wrist = keypoints[10]
    left_hip = keypoints[11]
    right_hip = keypoints[12]
    left_knee = keypoints[13]
    right_knee = keypoints[14]
    left_ankle = keypoints[15]
    right_ankle = keypoints[16]        
    
    color_center = (255, 255, 255)
    color_left = (255,255,0)
    color_right = (0,255,255)
    
    draw_valid_line(draw, left_shoulder, right_shoulder, color_center)
    draw_valid_line(draw, left_hip, right_hip, color_center)
    
    draw_valid_line(draw, nose, left_eye, color_left)
    draw_valid_line(draw, left_eye, left_ear, color_left)
    
    draw_valid_line(draw, nose, right_eye, color_right)
    draw_valid_line(draw, right_eye, right_ear, color_right)
        
    draw_valid_line(draw, left_shoulder, left_ear, color_left)    
    draw_valid_line(draw, left_shoulder, left_elbow, color_left)
    draw_valid_line(draw, left_shoulder, left_hip, color_left)
    draw_valid_line(draw, left_elbow, left_wrist, color_left)
    draw_valid_line(draw, left_hip, left_knee, color_left)
    draw_valid_line(draw, left_knee, left_ankle, color_left)
    
    draw_valid_line(draw, right_shoulder, right_ear, color_right)
    draw_valid_line(draw, right_shoulder, right_elbow, color_right)
    draw_valid_line(draw, right_shoulder, right_hip, color_right)
    draw_valid_line(draw, right_elbow, right_wrist, color_right)    
    draw_valid_line(draw, right_hip, right_knee, color_right)
    draw_valid_line(draw, right_knee, right_ankle, color_right)
    

In [ ]:
def draw_box_keypoint(image, keypoint_box_label, is_show_class=False):
    #keypoint_box_label : (m, 17*3+5)
    box_label = keypoint_box_label[:, :5]
    keypoint_label = keypoint_box_label[:,5:].reshape((-1, NUM_KEYPOINT, NUM_KEYPOINT_CH))
    #box_label : (m, 5)
    #keypoint_label : (m, 17, 3)
    
    if np.mean(box_label[:, :4]) < 2:
        print('unnormalize')
        img_w, img_h = image.width, image.height        
        box_label[:, :4] *= np.array((img_w, img_h, img_w, img_h)).reshape((1, 4))
        keypoint_label[:, :, :2] *= np.array((img_w, img_h)).reshape((1, 1, 2))
    
    for keypoints in keypoint_label:
        draw_keypoint(image, keypoints, is_show_class=is_show_class)
        draw_keypoint_line(image, keypoints)

    for cbox in box_label:
        draw_cbox(image, cbox)    

#### test

In [ ]:
sample_i = 9

In [ ]:
sample_image_path = list(dict_annotation_coco.keys())[sample_i]
keypoint_box_label_sample = dict_annotation_coco[sample_image_path]

box_label_sample = keypoint_box_label_sample[:, :5]
keypoint_label_sample = keypoint_box_label_sample[:,5:].reshape((-1, 17, 3))#todo 
sample_image_path, len(keypoint_label_sample), keypoint_label_sample

In [ ]:
Image.open(sample_image_path)

In [ ]:
pd.DataFrame(keypoint_label_sample[0].astype(np.int), class_keypoints, ['x','y','visibility'])

In [ ]:
box_label_sample.shape

In [ ]:
sample_image = Image.open(sample_image_path)
print('sample_image.size', sample_image.size)
draw_box_keypoint(sample_image, keypoint_box_label_sample, is_show_class=True)

In [ ]:
sample_image

- 학습 데이터 구성
- X : 이미지 (h,w,3)
- Y : 박스 + 클래스 + 키포인트 (box:4, class:1, keypoint:17*3(x,y,visibility)))

keypoint 는 영상에서의 좌표가 아닌 박스에서의 상대좌표? NO, 박스가 부정확하면 keypoint 까지 망가진다
- 박스 검출 후 그 내부에서 keypoint 의 상대좌표를 regression 하면 안됨

keypoint 를 검출해서 그것으로부터 박스를 그리는 것이 안전
- keypoint 의 visibility 까지 학습시켜서 keypoint를 그리는데 사용하자
- 그려진 keypoint를 이용해서 box 를 그리자

- box & keypoint = positive
- no box * no keypoint = background
- box & no keypoint = ignore (New!)
- no box & keypoint = not exist

1. keypoint는 groundtruth 로 클래스별 hitmap으로 변환해서 traget 이 되어 학습된다.
    - hitmap gt 는 좌표맵과 키포인트 좌표의 차이를 이용해 생성한다
      - 차이가 가장 작은 위치는 positive + (VISIBLE, OCCLUDED)
      - 차이가 일정 범위 이하는 ignore
      - 차이가 일정 범위 이상은 negative + (NOT_EXIST)
    - hitmap_visibility_gt 는 hitmap_coord_gt 와 별도로 생성한다
      - hitmap의 할당값은 VISIBLE(2), OCCLUDED(1), NOT_EXIST(0) 로 맵핑
      - hitmap_gt는 class별로 존재하나 visibility 는 class와 독립적인 변수이다.
      - hitmap_gt는 keypoint를 찾는 용도이며 visibility는 hitmap_gt로 찾은 keypoint의 상태이다.
    - hitmap gt 는 multi-scale 일 필요가 있다
    - hitmap gt 는 anchor별로 생성될 필요가 있다. 그것이 구현이 더 편리하다
    - 모델은 클래스별 hitmap을 출력한다
    - hitmap loss 적용
1. 모델은 검출기와 같은 anchor를 사용한다
   - 모델은 objectness와 박스 regression을 출력한다
1. positive objectness를 갖는 anchor 는 모델이 예측한 box 변환 값을 이용해 오브젝트 후보의 경계 박스(ROI)를 생성한다
   1. ROI 내부에 존재하는 최대 확률의 keypoint (ROI에 대한 상대)좌표를 구한다
      - keypoint 간의 연결성에 대해서는 무시하는 꼴이다.
      - 따라서 주위에서 침입한 keypoint 가 있는 경우 문제가 발생한다. 이 문제는 hierarchical keypoint 나 affinity vector를 이용하면 해결 가능하다.
      - 일단은 연결성은 무시해서 구현해보자
   1. ROI 정보를 이용해 상대좌표를 이미지에 대한 상대 좌표로 변환한다

In [ ]:
def load_xy(annotation, stride=1):
    input_list = []
    bbox_list = []
    path_list = []
    i = 0
    
    for path_image in annotation:
        i+=1
        if stride!=1 and np.random.randint(1, 1+stride)%stride!=0:
            continue
            
        cls_bbox = annotation[path_image]                
        bbox = np.array(cls_bbox[:, :4])
        cls = cls_bbox[:, 4:5]
        keypoint = cls_bbox[:, 5:].astype(np.float)
        keypoint_3d = np.reshape(keypoint, (-1, NUM_KEYPOINT, NUM_KEYPOINT_CH))
        
        img = Image.open(path_image)    
        scale = np.array((img.width, img.height, img.width, img.height))
        scale = np.reshape(scale, (1, 4)).astype(np.float)
        scale_keypoint = np.array((img.width, img.height, 1)).reshape((1,1,3))

        img_arr = np.array(img)
        if img_arr.ndim < 3: 
            print('gray image skip')
            continue #gray image skip
        
        try:
            std_v = np.std(img_arr)
            if std_v < 3:
                print('std_v', std_v)
                continue
        except:
            print('error', path_image)
            continue
        
        box_width = bbox[:, 2] - bbox[:, 0]
        box_height = bbox[:, 3] - bbox[:, 1]
        
        if np.min(box_width) < 1 or np.min(box_height) < 1:
            print('box_size < 1', box_width, box_height)#check
            continue
            
        bbox_norm = bbox.astype(np.float) / scale        
        keypoint_norm = keypoint_3d / scale_keypoint        
        keypoint_norm_2d = np.reshape(keypoint_norm, (-1, NUM_KEYPOINT * NUM_KEYPOINT_CH))
        
        cls_bbox_norm = np.concatenate((bbox_norm, cls, keypoint_norm_2d), axis=1)

        input_list.append(img_arr)
        bbox_list.append(cls_bbox_norm)
        path_list.append(path_image)
        if len(input_list)%100==0:        
            print(len(annotation), i, len(input_list))   
        if len(input_list) >= max_data_m:
            break       

    print(len(input_list), len(bbox_list))
    return input_list, bbox_list

In [ ]:
len(dict_annotation_coco)

In [ ]:
list_x, list_y = load_xy(dict_annotation_coco, stride=1)

In [ ]:
list_x[0].shape, list_y[0].shape

In [ ]:
plt.imshow(list_x[0])

In [ ]:
list_y[0].shape

In [ ]:
list_y[0][:, :4]

In [ ]:
list_y[0][:, 5:].reshape((-1, NUM_KEYPOINT, NUM_KEYPOINT_CH))

## debug

In [ ]:
list_y[0][:, 5:].reshape((-1, NUM_KEYPOINT, NUM_KEYPOINT_CH))[:,:,-1]

In [ ]:
#list_y[0][:, 5:].reshape((-1, NUM_KEYPOINT, NUM_KEYPOINT_CH))[:,:,-1] = 1+np.arange(NUM_KEYPOINT)

In [ ]:
list_y[0][:, 5:].reshape((-1, NUM_KEYPOINT, NUM_KEYPOINT_CH))[:,:,-1]

In [ ]:
def display_data(list_x, list_y, stride=1):
    
    for i in range(0, len(list_x), stride):
        x = list_x[i]
        y = list_y[i]
                
        img = Image.fromarray(x)
        kp = y[:, -NUM_KEYPOINT * NUM_KEYPOINT_CH:]        
        kp_cls = kp[:, 2::3]
        print(i, 'y', 'box', y[:, :4])
        #print('kp', kp)
        draw_box_keypoint(img, y, is_show_class=False)
        display(img)    

In [ ]:
display_data(list_x, list_y, stride=1)

## preprocessing for model

In [ ]:
#https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import Tensor
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.models import Model

In [ ]:
#https://github.com/tensorflow/addons/blob/v0.11.2/tensorflow_addons/image/__init__.py
from tensorflow_addons.image.color_ops import sharpness
from tensorflow_addons.image.filters import gaussian_filter2d
from tensorflow_addons.image.dense_image_warp import dense_image_warp

In [ ]:
tf.__version__, tf.test.is_gpu_available()

In [ ]:
padded_image_shape = (512, 512)#320, 512, 640
anchor_k = 9
num_classes = 3
level_start = 4
level_end = 8
l1 = 1e-9
activation = 'swish'#'selu' is not converted to tflite
kernel_init = tf.initializers.he_normal()
edgecolors = np.random.rand(num_classes, 3) 
edgecolors = np.minimum(edgecolors+0.1, 1.0)
path_weight = "weight/keypoint_efficientDet-D2"

In [ ]:
list_x_train = list_x
list_x_test = list_x
list_y_train = list_y
list_y_test = list_y
len(list_x_train)

#### model function

In [ ]:
def swap_xy(boxes):
    """Swaps order the of x and y coordinates of the boxes.
    Arguments:
      boxes: A tensor with shape `(num_boxes, 4)` representing bounding boxes.
    Returns:
      swapped boxes with shape same as that of boxes.
    """
    return tf.stack([boxes[:, 1], boxes[:, 0], boxes[:, 3], boxes[:, 2]], axis=-1)


def convert_to_xywh(boxes):
    """Changes the box format to center, width and height.
    Arguments:
      boxes: A tensor of rank 2 or higher with a shape of `(..., num_boxes, 4)`
        representing bounding boxes where each box is of the format
        `[xmin, ymin, xmax, ymax]`.
    Returns:
      converted boxes with shape same as that of boxes.
    """
    return tf.concat(
        [(boxes[..., :2] + boxes[..., 2:]) / 2.0, boxes[..., 2:] - boxes[..., :2]],
        axis=-1,
    )


def convert_to_corners(boxes):
    """Changes the box format to corner coordinates
    Arguments:
      boxes: A tensor of rank 2 or higher with a shape of `(..., num_boxes, 4)`
        representing bounding boxes where each box is of the format
        `[x, y, width, height]`.
    Returns:
      converted boxes with shape same as that of boxes.
    """
    return tf.concat(
        [boxes[..., :2] - boxes[..., 2:] / 2.0, boxes[..., :2] + boxes[..., 2:] / 2.0],
        axis=-1,
    )


"""
## Computing pairwise Intersection Over Union (IOU)
As we will see later in the example, we would be assigning ground truth boxes
to anchor boxes based on the extent of overlapping. This will require us to
calculate the Intersection Over Union (IOU) between all the anchor
boxes and ground truth boxes pairs.
"""

def compute_iou(boxes1, boxes2):#compute_iou(anchor_boxes, gt_boxes)
    """Computes pairwise IOU matrix for given two sets of boxes
    Arguments:
      boxes1: A tensor with shape `(N, 4)` representing bounding boxes
        where each box is of the format `[x, y, width, height]`.
        boxes2: A tensor with shape `(M, 4)` representing bounding boxes
        where each box is of the format `[x, y, width, height]`.
    Returns:
      pairwise IOU matrix with shape `(N, M)`, where the value at ith row
        jth column holds the IOU between ith box and jth box from
        boxes1 and boxes2 respectively.
    """
    boxes1_corners = convert_to_corners(boxes1)
    boxes2_corners = convert_to_corners(boxes2)
    lu = tf.maximum(boxes1_corners[:, None, :2], boxes2_corners[:, :2])
    rd = tf.minimum(boxes1_corners[:, None, 2:], boxes2_corners[:, 2:])
    intersection = tf.maximum(0.0, rd - lu)
    intersection_area = intersection[:, :, 0] * intersection[:, :, 1]
    boxes1_area = boxes1[:, 2] * boxes1[:, 3]
    boxes2_area = boxes2[:, 2] * boxes2[:, 3]
    union_area = tf.maximum(
        boxes1_area[:, None] + boxes2_area - intersection_area, 1e-8
    )
    return tf.clip_by_value(intersection_area / union_area, 0.0, 1.0)

In [ ]:

"""
## Implementing Anchor generator
Anchor boxes are fixed sized boxes that the model uses to predict the bounding
box for an object. It does this by regressing the offset between the location
of the object's center and the center of an anchor box, and then uses the width
and height of the anchor box to predict a relative scale of the object. In the
case of RetinaNet, each location on a given feature map has nine anchor boxes
(at three scales and three ratios).
"""
class AnchorBox:
    """Generates anchor boxes.
    This class has operations to generate anchor boxes for feature maps at
    strides `[8, 16, 32, 64, 128]`. Where each anchor each box is of the
    format `[x, y, width, height]`.
    Attributes:
      aspect_ratios: A list of float values representing the aspect ratios of
        the anchor boxes at each location on the feature map
      scales: A list of float values representing the scale of the anchor boxes
        at each location on the feature map.
      num_anchors: The number of anchor boxes at each location on feature map
      areas: A list of float values representing the areas of the anchor
        boxes for each feature map in the feature pyramid.
      strides: A list of float value representing the strides for each feature
        map in the feature pyramid.
    """

    def __init__(self):
        self.level_start = level_start
        self.level_end = level_end
        
        if anchor_k==9:
            self.aspect_ratios = [0.5, 1.0, 2.0]        
            self.scales = [2 ** x for x in [0, 1 / 3, 2 / 3]]
        else:
            self.aspect_ratios = [1.0]        
            self.scales = [2 ** x for x in [0]]
                
        self._num_anchors = len(self.aspect_ratios) * len(self.scales)
        self._strides = [2 ** i for i in range(self.level_start, self.level_end)]
        self._areas = [x ** 2 for x in [32.0, 64.0, 128.0, 196.0, 256.0]]                        
        self._areas = self._areas[:level_end - level_start]
        
        self._anchor_dims = self._compute_dims()

    def _compute_dims(self):
        """Computes anchor box dimensions for all ratios and scales at all levels
        of the feature pyramid.
        """
        anchor_dims_all = []
        for area in self._areas:
            anchor_dims = []
            for ratio in self.aspect_ratios:
                anchor_height = tf.math.sqrt(area / ratio)
                anchor_width = area / anchor_height
                dims = tf.reshape(
                    tf.stack([anchor_width, anchor_height], axis=-1), [1, 1, 2]
                )
                for scale in self.scales:
                    anchor_dims.append(scale * dims)
            anchor_dims_all.append(tf.stack(anchor_dims, axis=-2))
        return anchor_dims_all

    def _get_anchors(self, feature_height, feature_width, level):
        """Generates anchor boxes for a given feature map size and level
        Arguments:
          feature_height: An integer representing the height of the feature map.
          feature_width: An integer representing the width of the feature map.
          level: An integer representing the level of the feature map in the
            feature pyramid.
        Returns:
          anchor boxes with the shape
          `(feature_height * feature_width * num_anchors, 4)`
        """
        rx = tf.range(feature_width, dtype=tf.float32) + 0.5
        ry = tf.range(feature_height, dtype=tf.float32) + 0.5
        centers = tf.stack(tf.meshgrid(rx, ry), axis=-1) * self._strides[level - self.level_start]
        centers = tf.expand_dims(centers, axis=-2)
        centers = tf.tile(centers, [1, 1, self._num_anchors, 1])
        dims = tf.tile(
            self._anchor_dims[level - self.level_start], [feature_height, feature_width, 1, 1]
        )
        anchors = tf.concat([centers, dims], axis=-1)
        return tf.reshape(
            anchors, [feature_height * feature_width * self._num_anchors, 4]
        )

    def get_anchors(self, image_height, image_width):
        """Generates anchor boxes for all the feature maps of the feature pyramid.
        Arguments:
          image_height: Height of the input image.
          image_width: Width of the input image.
        Returns:
          anchor boxes for all the feature maps, stacked as a single tensor
            with shape `(total_anchors, 4)`
        """
        anchors = [
            self._get_anchors(
                tf.math.ceil(image_height / 2 ** i),
                tf.math.ceil(image_width / 2 ** i),
                i,
            )
            for i in range(self.level_start, self.level_end)
        ]
        return tf.concat(anchors, axis=0)
    
    def get_anchors_check(self, image_height, image_width):
        """Generates anchor boxes for all the feature maps of the feature pyramid.
        Arguments:
          image_height: Height of the input image.
          image_width: Width of the input image.
        Returns:
          anchor boxes for all the feature maps, stacked as a single tensor
            with shape `(total_anchors, 4)`
        """
        anchors = [
            self._get_anchors(
                tf.math.ceil(image_height / 2 ** i),
                tf.math.ceil(image_width / 2 ** i),
                i,
            )
            for i in range(self.level_start, self.level_end)
        ]
        return anchors

In [ ]:
padded_image_shape

In [ ]:
def shuffle_pixel(net):
    # net # (m, h, w, c)    
    n0, n1, n2, n3 = tf.split(net, 4, -1)
    n01 = tf.stack((n0, n1), 3) # (m, h, w, 2, c)
    n23 = tf.stack((n2, n3), 3) # (m, h, w, 2, c)

    n01234 = tf.stack((n01, n23), 2)# (m, h, 2, w, 2, c)
    out = tf.reshape(n01234, [-1])#seg
    return out


def shuffle_pixel_2x(net, h, w, c):
    # net # (m, h, w, c)
    #n0, n1 = tf.split(net, 2, -1)
    n0, n1, n2, n3 = tf.split(net, 4, -1)
    n01 = tf.stack((n0, n1), 3) # (m, h, w, 2, c)
    n23 = tf.stack((n2, n3), 3) # (m, h, w, 2, c)

    n01234 = tf.stack((n01, n23), 2)# (m, h, 2, w, 2, c)
    out = tf.reshape(n01234, [-1, h*2, w*2, c//4])
    return out


def shuffle_pixel_3x(net, h, w):
    # net # (m, h, w, c)
    #n0, n1 = tf.split(net, 2, -1)
    n0, n1, n2, n3, n4, n5, n6, n7, n8 = tf.split(net, 9, -1)
    r0 = tf.stack((n0, n4, n1), 3) # (m, h, w, 2, c)
    r1 = tf.stack((n5, n6, n7), 3) # (m, h, w, 2, c)
    r2 = tf.stack((n2, n8, n3), 3) # (m, h, w, 2, c)

    r = tf.stack((r0, r1, r2), 2)# (m, h, 2, w, 2, c)
    out = tf.reshape(r, [-1, h*3, w*3, 1])
    return out


def shuffle_pixel_4x(net):
    # net # (m, h, w, c)
    #n0, n1 = tf.split(net, 2, -1)
    net_split = tf.split(net, 16, -1)
    r0 = tf.stack(net_split[0:4], 3) # (m, h, w, 2, c)
    r1 = tf.stack(net_split[4:8], 3) # (m, h, w, 2, c)
    r2 = tf.stack(net_split[8:12], 3)
    r3 = tf.stack(net_split[12:16], 3)

    r = tf.stack((r0, r1, r2, r3), 2)# (m, h, 4, w, 2, c)
    #out = tf.reshape(r, [-1, h*4, w*4, 1])
    out = tf.reshape(r, [-1])
    return out


def shuffle_pixel_5x(net, h, w, c=1):
    # net # (m, h, w, c)
    k = 5
    net_split = tf.split(net, k*k, -1)
    r0 = tf.stack(net_split[k * 0:k * 1], 3)
    r1 = tf.stack(net_split[k * 1:k * 2], 3)
    r2 = tf.stack(net_split[k * 2:k * 3], 3)
    r3 = tf.stack(net_split[k * 3:k * 4], 3)
    r4 = tf.stack(net_split[k * 4:k * 5], 3)

    r = tf.stack((r0, r1, r2, r3, r4), 2)
    out = tf.reshape(r, [-1, h*k, w*k, c])
    return out

In [ ]:
def image_color_augment(x):
    if tf.random.uniform(()) < -0.5:
        x_max = tf.reduce_max(x, [1, 2], True)
        x = x_max - x
    if tf.random.uniform(()) < -0.2:
        r, g, b = tf.split(x, 3, axis=-1)
        x = tf.concat((r, b, g), -1)
    elif tf.random.uniform(()) < -0.4:
        r, g, b = tf.split(x, 3, axis=-1)
        x = tf.concat((b, r, g), -1)
    if tf.random.uniform(()) < 0.2:
        x = tf.image.random_hue(x, 0.08)
        x = tf.image.random_saturation(x, 0.6, 1.6)
    if tf.random.uniform(()) < 0.2:
        x = tf.image.random_brightness(x, 0.05)
        x = tf.image.random_contrast(x, 0.7, 1.3)
    if tf.random.uniform(()) < -0.2:
        gray = tf.image.rgb_to_grayscale(x)
        x = tf.concat((gray, gray, gray), -1)        
   
    if tf.random.uniform(()) < -0.2:
        x = gaussian_filter2d(x, filter_shape=tuple(np.random.randint(1, 10, (2))), sigma=10)
        #x = gaussian_filter2d(x, filter_shape=np.random.randint(3, 10, (2)), sigma=10)
    if tf.random.uniform(()) < 0.2:        
        x = sharpness(x, factor=10)    
    
    return x

In [ ]:
def random_flip_horizontal(image, boxes):
    """Flips image and boxes horizontally with 50% chance
    Arguments:
      image: A 3-D tensor of shape `(height, width, channels)` representing an
        image.
      boxes: A tensor with shape `(num_boxes, 4)` representing bounding boxes,
        having normalized coordinates.
    Returns:
      Randomly flipped image and boxes
    """
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        boxes = tf.stack([1 - boxes[:, 2], boxes[:, 1], 1 - boxes[:, 0], boxes[:, 3]], axis=-1)
   
    return image, boxes

def resize_and_pad_image(
    image, min_side=512.0, jitter=[128*4, 128*4+1], stride=128.0):
   
    image_shape = tf.cast(tf.shape(image)[:2], dtype=tf.float32)
    
    ratio = min_side / tf.reduce_min(image_shape)
    
    image_shape = ratio * image_shape
    image = tf.image.resize(image, tf.cast(image_shape, dtype=tf.int32))
    
    image = tf.image.pad_to_bounding_box(image, 0, 0, padded_image_shape[0], padded_image_shape[1]) 
    
    return image, image_shape, ratio

def resize_and_pad_image_bbox(
    image, bbox, mask_obj=None, min_side=1024.0, max_side=1024.0*4, jitter=[128*7+32, 128*8-32], stride=128.0
):
    #image, min_side=800.0, max_side=1333.0, jitter=[640, 1024], stride=128.0
    """Resizes and pads image while preserving aspect ratio.
    1. Resizes images so that the shorter side is equal to `min_side`
    2. If the longer side is greater than `max_side`, then resize the image
      with longer side equal to `max_side`
    3. Pad with zeros on right and bottom to make the image shape divisible by
    `stride`
    Arguments:
      image: A 3-D tensor of shape `(height, width, channels)` representing an
        image.
      min_side: The shorter side of the image is resized to this value, if
        `jitter` is set to None.
      max_side: If the longer side of the image exceeds this value after
        resizing, the image is resized such that the longer side now equals to
        this value.
      jitter: A list of floats containing minimum and maximum size for scale
        jittering. If available, the shorter side of the image will be
        resized to a random value in this range.
      stride: The stride of the smallest feature map in the feature pyramid.
        Can be calculated using `image_size / feature_map_size`.
    Returns:
      image: Resized and padded image.
      image_shape: Shape of the image before padding.
      ratio: The scaling factor used to resize the image
    """
    image_shape = tf.cast(tf.shape(image)[:2], dtype=tf.float32)
    if jitter is not None:
        min_side = tf.random.uniform((), jitter[0], jitter[1], dtype=tf.float32)
    ratio = min_side / tf.reduce_min(image_shape)
    if ratio * tf.reduce_max(image_shape) > max_side:
        ratio = max_side / tf.reduce_max(image_shape)
    image_shape = ratio * image_shape
    ratio_jitter = tf.random.uniform(tf.shape(image_shape), -32, 32, dtype=tf.float32)
    image_shape += ratio_jitter      
    image = tf.image.resize(image, tf.cast(image_shape, dtype=tf.int32))
    if mask_obj!=None:
        mask_obj = tf.image.resize(mask_obj, tf.cast(image_shape, dtype=tf.int32))
    padded_image_shape = tf.cast(
        tf.math.ceil(image_shape / stride) * stride, dtype=tf.int32
    )
    image = tf.image.pad_to_bounding_box(image, 0, 0, padded_image_shape[0], padded_image_shape[1])
    if mask_obj!=None:
        mask_obj = tf.image.pad_to_bounding_box(mask_obj, 0, 0, padded_image_shape[0], padded_image_shape[1])        
    padded_image_shape = tf.cast(padded_image_shape, tf.float32)              
    pad_ratio = tf.cast(image_shape, tf.float32) / padded_image_shape
    bbox_padded = tf.stack(
        [
            bbox[:, 0] * pad_ratio[1],
            bbox[:, 1] * pad_ratio[0],
            bbox[:, 2] * pad_ratio[1],
            bbox[:, 3] * pad_ratio[0],
        ],
        axis=-1,
    )
    if mask_obj!=None:
        return image, padded_image_shape, ratio, bbox_padded, mask_obj    
    return image, padded_image_shape, ratio, bbox_padded



In [ ]:
"""
## Preprocessing data
Preprocessing the images involves two steps:
- Resizing the image: Images are resized such that the shortest size is equal
to 800 px, after resizing if the longest side of the image exceeds 1333 px,
the image is resized such that the longest size is now capped at 1333 px.
- Applying augmentation: Random scale jittering  and random horizontal flipping
are the only augmentations applied to the images.
Along with the images, bounding boxes are rescaled and flipped if required.
"""

def unnormalize_box(bbox, image_shape):
    img_h = image_shape[0]
    img_w = image_shape[1]
    bbox = tf.stack(
        [
            bbox[:, 0] * img_w,
            bbox[:, 1] * img_h,
            bbox[:, 2] * img_w,
            bbox[:, 3] * img_h,
        ],
        axis=-1,
    )
    bbox = convert_to_xywh(bbox)    
    return bbox    


def unnormalize_keypoint(keypoint, image_shape):
    img_h = image_shape[0]
    img_w = image_shape[1]
    keypoint_un = tf.stack(
        [
            keypoint[:, :, 0] * img_w,
            keypoint[:, :, 1] * img_h,
            keypoint[:, :, 2]
        ],
        axis=-1,
    )    
    
    keypoint_un = tf.cast(keypoint_un, tf.float32)
    return keypoint_un


def resize_image(image):
    image_shape = tf.cast(tf.shape(image)[0:2], tf.float32)    
    resized_ratio = tf.cast(padded_image_shape / image_shape, tf.float32)
    
    image = tf.image.resize(image, padded_image_shape)    
    return image, resized_ratio


def preprocess_data(image, label):     
     
    #label (m,56)
    bbox_norm = label[:, :4]    
    cls = label[:, 4]
    keypoints = label[:, 5:]
    keypoint_3d = tf.reshape(keypoints, [-1, NUM_KEYPOINT, NUM_KEYPOINT_CH])    
    keypoint_3d = tf.cast(keypoint_3d, tf.float32)

    #image, bbox = random_flip_horizontal(image, bbox)        
    #image, image_shape, _, bbox = resize_and_pad_image_bbox(image, bbox)    
    #image, image_shape, _ = resize_and_pad_image(image)
    
    image, image_shape = resize_image(image)
    bbox_unnorm = unnormalize_box(bbox_norm, image_shape)
    keypoints_unnorm = unnormalize_keypoint(keypoint_3d, image_shape)
    #print('keypoints_unnorm', keypoints_unnorm)
    
    return image, bbox_unnorm, cls, keypoints_unnorm
    

In [ ]:
channel_label, NUM_KEYPOINT, NUM_KEYPOINT_CH

In [ ]:
def generator():
    #ind = np.arange((len(list_x_train)//2)*2)
    #ind = np.arange(len(list_x_train))    
    for i in range(len(list_x_train)):
        x = list_x_train[i]
        y = list_y_train[i]        
        yield (x, y)

def generator_test():    
    for i in range(len(list_x_test)):
        x = list_x_test[i]
        y = list_y_test[i]        
        yield (x, y)

dataset = tf.data.Dataset.from_generator(
    generator, 
    output_types=(tf.uint8, tf.float32), 
    output_shapes=(tf.TensorShape([None, None, 3]), tf.TensorShape([None, channel_label])))
dataset_test = tf.data.Dataset.from_generator(
    generator_test, 
    output_types=(tf.uint8, tf.float32), 
    output_shapes=(tf.TensorShape([None, None, 3]), tf.TensorShape([None, channel_label])))

np.set_printoptions(precision=2)
for example in tfds.as_numpy(dataset):
    image = example[0]
    label = example[1]
    print(image.dtype, image.shape, label.shape)
    print(label[0])
    #print(label[0][-NUM_KEYPOINT*3::3])
    break

In [ ]:
#전처리 확인
train_dataset = dataset.map(preprocess_data)
for x, yb, yc, yk in train_dataset:
    print('yb', yb)
    print('yc', yc)
    print('yk', yk)
    break

In [ ]:
def coordinate_map_uv(h, w):
    #return (6, 18, 256)
    x = tf.range(0.5, w, 1) / tf.cast(w, tf.float32) * 2.0 -1
    y = tf.range(0.5, h, 1) / tf.cast(h, tf.float32) * 2.0 -1    
    X, Y = tf.meshgrid(x, y)
    xy = tf.stack((X, Y), -1)
    xy = tf.expand_dims(xy, axis=0)   
    return xy 

def coordinate_map(h, w):
    #return (1, h, w, 2)    
    x = tf.range(w)
    y = tf.range(h)
    X, Y = tf.meshgrid(x, y)
    xy = tf.stack((X, Y), -1)
    xy = tf.cast(xy, tf.float32)
    return xy 

def add_map(net):
    shape = tf.shape(net)
    map_norm = coordinate_map(shape[1], shape[2])        
    map_square = tf.sqrt(map_norm)
    net = tf.concat((map_norm + net[:, :, :, :2], net[:, :, :, 2:]), -1)    
    return net

In [ ]:
"""
## Encoding labels
The raw labels, consisting of bounding boxes and class ids need to be
transformed into targets for training. This transformation consists of
the following steps:
- Generating anchor boxes for the given image dimensions
- Assigning ground truth boxes to the anchor boxes
- The anchor boxes that are not assigned any objects, are either assigned the
background class or ignored depending on the IOU
- Generating the classification and regression targets using anchor boxes
"""

class LabelEncoder:
    """Transforms the raw labels into targets for training.
    This class has operations to generate targets for a batch of samples which
    is made up of the input images, bounding boxes for the objects present and
    their class ids.
    Attributes:
      anchor_box: Anchor box generator to encode the bounding boxes.
      box_variance: The scaling factors used to scale the bounding box targets.
    """

    def __init__(self):
        self._anchor_box = AnchorBox()
        self._box_variance = tf.convert_to_tensor(
            [0.1, 0.1, 0.2, 0.2], dtype=tf.float32
        )           
    
    def _match_anchor_boxes(
        self, anchor_boxes, gt_boxes, match_iou=0.3, ignore_iou=0.1
    ):
        # anchor : (anchor_k, 4), gt_boxes : (box_m, 4)
        iou_matrix = compute_iou(anchor_boxes, gt_boxes)#(anchor_k, box_m)        
        max_iou = tf.reduce_max(iou_matrix, axis=1)#from anchor to object-box        
        matched_gt_idx = tf.argmax(iou_matrix, axis=1)    
        positive_mask = tf.greater_equal(max_iou, match_iou)# not only this, but also need max iou cell
        
        positive_proposal_mask = tf.greater_equal(iou_matrix, match_iou)
        positive_mask = tf.reduce_any(positive_proposal_mask, axis=1)        
        negative_mask = tf.less(max_iou, ignore_iou)
        
        max_iou_anchor = tf.reduce_max(iou_matrix, axis=0, keepdims=True)
        max_iou_anchor_mask = tf.greater_equal(iou_matrix, max_iou_anchor)
        
        positive_max_mask = tf.reduce_any(max_iou_anchor_mask, axis=1)
        positive_mask = tf.logical_or(positive_mask, positive_max_mask)#new      
        
        negative_mask = tf.logical_and(negative_mask, tf.logical_not(positive_mask))
        ignore_mask = tf.logical_not(tf.logical_or(positive_mask, negative_mask))        
        
        return (
            matched_gt_idx,            
            tf.cast(positive_mask, dtype=tf.float32),
            tf.cast(positive_max_mask, dtype=tf.float32),            
            tf.cast(ignore_mask, dtype=tf.float32),
        )
    
    
    def _compute_box_target(self, anchor_boxes, matched_gt_boxes):
        """Transforms the ground truth boxes into targets for training"""
        target = tf.concat(
            [
                (matched_gt_boxes[:, :2] - anchor_boxes[:, :2]) / anchor_boxes[:, 2:],
                tf.math.log(matched_gt_boxes[:, 2:] / anchor_boxes[:, 2:]),
            ],
            axis=-1,
        )
        target = target / self._box_variance
        return target
    
    def _compute_keypoint_target(self, anchor_boxes, matched_gt_points):
        """Transforms the ground truth boxes into targets for training"""
        target = (matched_gt_points[:, :, :2] - anchor_boxes[:, None, :2]) / anchor_boxes[:, None, 2:]        
        target = target / self._box_variance[:2]
        return target    
      
    
    def _encode_sample(self, image_shape, gt_boxes, cls_ids, gt_keypoints):
        """Creates box and classification targets for a single sample"""
        
        anchor_boxes = self._anchor_box.get_anchors(image_shape[1], image_shape[2])
        
        matched_gt_idx, positive_mask, positive_max_mask, ignore_mask = self._match_anchor_boxes(
            anchor_boxes, gt_boxes)
        
        gt_keypoints_xy = gt_keypoints[:, :, :2]
        gt_keypoints_cls = gt_keypoints[:, :, 2]       
        
        print('gt_keypoints_xy', gt_keypoints_xy)
        
        matched_gt_boxes = tf.gather(gt_boxes, matched_gt_idx)
        matched_gt_keypoint_xy = tf.gather(gt_keypoints_xy, matched_gt_idx)
        
        box_target = self._compute_box_target(anchor_boxes, matched_gt_boxes)
        keypoint_xy_target = self._compute_keypoint_target(anchor_boxes, matched_gt_keypoint_xy)
                
        matched_gt_cls_ids = tf.gather(cls_ids, matched_gt_idx)
        matched_gt_keypoint_cls = tf.gather(gt_keypoints_cls, matched_gt_idx)
                
        cls_target = tf.where(tf.not_equal(positive_mask, 1.0), 0.0, matched_gt_cls_ids)        
        cls_target = tf.where(tf.equal(ignore_mask, 1.0), -1.0, cls_target)
                
        #[?,4], [?,?], [?,17,2], []
        keypoint_xy_target = tf.reshape(keypoint_xy_target, [-1, NUM_KEYPOINT * 2])
        keypoint_cls_target = tf.reshape(matched_gt_keypoint_cls, [-1, NUM_KEYPOINT])
                
        cls_target = tf.expand_dims(cls_target, axis=-1)
        targets = tf.concat([box_target, cls_target, keypoint_xy_target, keypoint_cls_target], axis=-1)
        return targets
    
    
    def _encode_keypoint_heatmap(self, img_h, img_w, keypoints):
        #keypoints (n, 17, 3)
        #return gt_map (h, w)        
        neighbor_k = 4
        keypoints_xy = keypoints[:, :, :2]
        keypoints_visible = keypoints[:, :, 2]
        keypoints_cls = tf.zeros_like(keypoints_visible) + tf.range(NUM_KEYPOINT, dtype=tf.float32)
                
        xy_map = coordinate_map(img_h, img_w) + 0.5                
        
        keypoints_xy_5d = tf.reshape(keypoints_xy, [1, 1, -1, NUM_KEYPOINT, 2])#(1, 1, n, 17, 2)        
        keypoints_visible_5d = tf.reshape(keypoints_visible, [1, 1, -1, NUM_KEYPOINT, 1])
        keypoints_visible_4d_mask = tf.cast(keypoints_visible_5d[:,:,:,:,0] > 1, tf.float32)
                
        keypoints_xy_5d_to_find_min = keypoints_xy_5d + 5000 * tf.cast(keypoints_visible_5d < 1,tf.float32)
        visible_max_xy = tf.reduce_max(keypoints_xy_5d, 3, True)
        visible_min_xy = tf.reduce_min(keypoints_xy_5d_to_find_min, 3, True)
        visible_xy_range = tf.abs(visible_max_xy - visible_min_xy)#(1, 1, n, 1, 2)
        
        neighbor_k_relative = neighbor_k + 0.04 * tf.reduce_mean(visible_xy_range, -1)#(1, 1, n, 1)
        neighbor_k_relative = neighbor_k_relative + (4 * keypoints_visible_4d_mask)
        
        xy_map_exp = tf.expand_dims(tf.expand_dims(xy_map, 2), 2)
        distance_xy = xy_map_exp - keypoints_xy_5d#(h, w, 1, 1, 2) - (1, 1, n, 17, 2)               
        distance = tf.sqrt(tf.reduce_sum(tf.square(distance_xy), -1))#(h, w, n, 17)
        
        #[?,?,n,17], [1,1,n,1]                
        disk_mask = tf.cast(distance < neighbor_k_relative, tf.float32) * keypoints_visible_4d_mask
        bg_zero = tf.zeros_like(distance[:,:,:,:1])
                
        disk_exist = tf.reduce_sum(disk_mask, [2, 3], True) > 0
        disk_exist_mask = tf.cast(disk_exist, tf.float32)
        bg_map = disk_exist_mask * 10000 + (1 - disk_exist_mask) * (bg_zero - 10000)
        
        distance_map_with_bg = tf.concat((bg_map, distance), -1)        
        distance_map_with_bg = tf.reshape(distance_map_with_bg, [img_h, img_w, -1]) 
        
        cls_map = tf.argmin(distance_map_with_bg, -1, output_type=tf.int32)#(h, w, n)                
        cls_k = NUM_KEYPOINT + 1
        cls_map = cls_map % cls_k        
        return cls_map
    
    def encode_batch(self, batch_images, gt_boxes, cls_ids, keypoints):
        """Creates box and classification targets for a batch"""
        #keypoints(batch_m, n, 17, 3)
        images_shape = tf.shape(batch_images)
        batch_size = images_shape[0]
        
        labels = tf.TensorArray(dtype=tf.float32, size=batch_size, dynamic_size=True)
        labels_heatmap = tf.TensorArray(dtype=tf.int32, size=batch_size, dynamic_size=True)
        for i in range(batch_size):
            label = self._encode_sample(images_shape, gt_boxes[i], cls_ids[i], keypoints[i])
            labels = labels.write(i, label)
            label_heat = self._encode_keypoint_heatmap(images_shape[1], images_shape[2], keypoints[i])        
            labels_heatmap = labels_heatmap.write(i, label_heat)
        
        batch_images = tf.cast(batch_images, tf.float32)
        labels = labels.stack()
        labels_heatmap = labels_heatmap.stack()
        
        #return batch_images, labels, labels_hitmap
        return batch_images, {"detect": labels, "heatmap": labels_heatmap}#dual
     
    def encode_batch_train(self, batch_images, gt_boxes, cls_ids, keypoints):
        
        batch_images = image_color_augment(batch_images)
        return self.encode_batch(batch_images, gt_boxes, cls_ids, keypoints)    

In [ ]:
level_start, 2**level_start

In [ ]:
label_encoder = LabelEncoder()
anchor_k = len(label_encoder._anchor_box.aspect_ratios)*len(label_encoder._anchor_box.scales)
anchor_k

In [ ]:
strategy = tf.distribute.MirroredStrategy()
#strategy = tf.distribute.OneDeviceStrategy(device='/gpu:0')
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

In [ ]:
batch_size = min(strategy.num_replicas_in_sync, len(list_x_train))
autotune = tf.data.experimental.AUTOTUNE
batch_size

In [ ]:
check_dataset = dataset.map(preprocess_data, num_parallel_calls=autotune)
print('check_dataset', check_dataset)
for image, bbox_unnorm, cls, keypoints_unnorm in check_dataset:
    print(image.shape, bbox_unnorm.shape, cls.shape, keypoints_unnorm.shape)
    print('bbox_unnorm', bbox_unnorm)
    print('cls', cls)
    print('keypoints_unnorm', keypoints_unnorm)
    break

In [ ]:
train_dataset = dataset.map(preprocess_data, num_parallel_calls=autotune)
#train_dataset = train_dataset.shuffle(8 * batch_size)
#train_dataset = train_dataset.padded_batch(batch_size=batch_size, padding_values=(0.0, 1e-8, -1), drop_remainder=False)
train_dataset = train_dataset.padded_batch(batch_size=batch_size)
train_dataset = train_dataset.map(label_encoder.encode_batch_train, num_parallel_calls=autotune)
train_dataset = train_dataset.prefetch(autotune)

In [ ]:
for x, multi_y in train_dataset:
    #print('multi_y', type(multi_y), multi_y.keys())
    y = multi_y['detect']
    y_heatmap = multi_y['heatmap']
    #print('x', x.shape)
    #print('y', y.shape)
    #print('heatmap', heatmap.shape)
    print('heatmap.max()', tf.shape(y_heatmap), tf.reduce_max(y_heatmap))
    
    img_map_xy = tf.cast(y_heatmap[0], tf.uint8)
    alpha = 255//(NUM_KEYPOINT+1)
    hitmap_h = img_map_xy * alpha
    hitmap_img = tf.stack((hitmap_h, hitmap_h, hitmap_h), -1)
    fig, ax = plt.subplots(1, 2)    
    ax[0].imshow(tf.cast(x[0], tf.uint8))
    ax[1].imshow(hitmap_h, cmap='gray')    
    #break

In [ ]:
val_dataset = dataset_test.map(preprocess_data, num_parallel_calls=autotune)
#val_dataset = val_dataset.padded_batch(batch_size=batch_size, padding_values=(0.0, 1e-8, -1), drop_remainder=False)
val_dataset = val_dataset.padded_batch(batch_size=1)
val_dataset = val_dataset.map(label_encoder.encode_batch, num_parallel_calls=autotune)
val_dataset = val_dataset.prefetch(autotune)

In [ ]:
a = tf.cast([[1,2]], tf.float32)
b = tf.constant((1,4))
tf.tile(a, b)

In [ ]:
a = tf.cast([[[1,2]]], tf.float32)
b = tf.constant((1,1, 4))
tf.tile(a, b)

In [ ]:
def split_gt(y_true):
    #targets = tf.concat([box_target, cls_target, keypoint_xy_target, keypoint_cls_target], axis=-1)
    y_box = y_true[:, :, :4]
    y_cls = y_true[:, :, 4]
    y_keypoint = y_true[:, :, 5:]
    return y_box, y_cls, y_keypoint

In [ ]:
#filters = 4 + 1 + num_classes + num_keypoints * (2 + num_keypoint_classes)       
def split_hyperthesis(y_pred):
    h_box = y_pred[:, :, :4]
    h_obj = y_pred[:, :, 4]
    h_cls = y_pred[:, :, 5:5+num_classes]        
    h_keypoint = y_pred[:, :, 5+num_classes:]       
    
    h_obj = tf.nn.sigmoid(h_obj)
    return h_box, h_obj, h_cls, h_keypoint

In [ ]:
def split_gt_keypoint(y_keypoint):
    xy = y_keypoint[:, :, :NUM_KEYPOINT*2]
    cls = y_keypoint[:, :, -NUM_KEYPOINT:]
        
    x = xy[:, :, 0::2]
    y = xy[:, :, 1::2]    
    return x, y, cls    

In [ ]:
def split_h_keypoint(h_keypoint):    
    h_x = h_keypoint[:, :, :NUM_KEYPOINT]
    h_y = h_keypoint[:, :, NUM_KEYPOINT:NUM_KEYPOINT * 2]
    h_cls_score = h_keypoint[:, :, NUM_KEYPOINT * 2:]
    
    #h_x = tf.tanh(h_x)
    #h_y = tf.tanh(h_y)
    
    keypoints_cls = tf.round(tf.sigmoid(h_cls_score) * 2)
    return h_x, h_y, h_cls_score, keypoints_cls

In [ ]:
def _decode_box_predictions(anchor_boxes, box_predictions):        
    boxes = box_predictions * label_encoder._box_variance
    boxes = tf.concat(
        [
            boxes[:, :, :2] * anchor_boxes[:, :, 2:] + anchor_boxes[:, :, :2],
            tf.math.exp(boxes[:, :, 2:]) * anchor_boxes[:, :, 2:],
        ],
        axis=-1,
    )
    boxes_transformed = convert_to_corners(boxes)
    return boxes_transformed

def _decode_keypoint_predictions(anchor_boxes, keypoint_predictions):        
    repeat = tf.constant([NUM_KEYPOINT], tf.int32)
    box_variance_rep = tf.tile(label_encoder._box_variance[:2], repeat)
    keypoints = keypoint_predictions * box_variance_rep
    
    batch_m = tf.shape(keypoints)[0]
    repeat = tf.constant((1, 1, NUM_KEYPOINT))
    
    anchor_xy_tile = tf.tile(anchor_boxes[:, :, :2], repeat)
    anchor_wh_tile = tf.tile(anchor_boxes[:, :, 2:], repeat)
    
    keypoint = keypoints * anchor_wh_tile + anchor_xy_tile
    return keypoint


def decode_debug(images, predictions, 
                      num_classes=num_classes,
                      confidence_threshold=0.5,
                      nms_iou_threshold=0.2,
                      max_detections_per_class=1000,
                      max_detections=1500,
                      box_variance=[0.1, 0.1, 0.2, 0.2]):
    
    _anchor_box = AnchorBox()    
        
    image_shape = tf.cast(tf.shape(images), dtype=tf.float32)
    #anchor_boxes = _anchor_box.get_anchors(image_shape[1], image_shape[2])#free size        
    image_h = padded_image_shape[0]
    image_w = padded_image_shape[1]
    anchor_boxes = _anchor_box.get_anchors(image_h, image_w)
    
    box_predictions, objectness, keypoint_predictions = split_gt(predictions)
    x, y, keypoints_cls = split_gt_keypoint(keypoint_predictions)
    
    m = tf.shape(images)[0]
    keypoints_xy = tf.stack((x, y), -1)
    keypoints_xy = tf.reshape(keypoints_xy, (m, -1, NUM_KEYPOINT * 2))
       
    cls_score = predictions[:, :, 5:5+num_classes]    
    cls_prob = tf.nn.softmax(cls_score)
    cls_prob_max = tf.reduce_max(cls_prob, -1)
    cls = tf.argmax(cls_score, -1)
    cls = tf.cast(cls, tf.float32)
    cls = predictions[:, :, 4]
            
    boxes = _decode_box_predictions(anchor_boxes[None, ...], box_predictions)
    keypoints_xy = _decode_keypoint_predictions(anchor_boxes[None, ...], keypoints_xy)
    
    boxes_2d = tf.reshape(boxes, [-1, 4])    
    scores = tf.reshape(objectness, [-1, 1])
    
    cls = tf.reshape(cls, [-1, 1])
    
    keypoints_xy_3d = tf.reshape(keypoints_xy, [-1, NUM_KEYPOINT, 2])
    keypoints_cls_3d = tf.reshape(keypoints_cls, [-1, NUM_KEYPOINT, 1])    
    keypoints_3d = tf.concat((keypoints_xy_3d, keypoints_cls_3d), -1)
    keypoints_2d = tf.reshape(keypoints_3d, [-1, NUM_KEYPOINT * 3])
    
    ccbox = tf.concat((cls, scores, boxes_2d, keypoints_2d), -1)
    ccbox_check = tf.concat((boxes_2d, cls, keypoints_2d), -1)
    
    selected_indices, selected_scores = tf.image.non_max_suppression_with_scores(    
        ccbox[:, 2:2+4],
        ccbox[:, 1],        
        max_detections,
        nms_iou_threshold,
        confidence_threshold,        
    )
    output = tf.gather(ccbox_check, selected_indices)        
    return output   

In [ ]:
def decodePredictions(images, predictions, 
                      num_classes=num_classes,
                      confidence_threshold=0.5,
                      nms_iou_threshold=0.2,
                      max_detections_per_class=100,
                      max_detections=150,
                      box_variance=[0.1, 0.1, 0.2, 0.2]):
    
    _anchor_box = AnchorBox()    
        
    image_shape = tf.cast(tf.shape(images), dtype=tf.float32)
    anchor_boxes = _anchor_box.get_anchors(image_shape[1], image_shape[2])#free size        
    #image_h = padded_image_shape[0]
    #image_w = padded_image_shape[1]
    #anchor_boxes = _anchor_box.get_anchors(image_h, image_w)
   
    h_box, objectness, h_cls, h_keypoint = split_hyperthesis(predictions)    
    h_x, h_y, h_cls_score, keypoints_cls = split_h_keypoint(h_keypoint)
    
    keypoints_xy = tf.stack((h_x, h_y), -1) #keypoint_predictions[:, :, :NUM_KEYPOINT * 2]
    m = tf.shape(images)[0]
    
    keypoints_xy = tf.reshape(keypoints_xy, [m, -1, NUM_KEYPOINT * 2])
    
    cls_score = predictions[:, :, 5:5+num_classes]    
    cls_prob = tf.nn.softmax(cls_score)
    cls_prob_max = tf.reduce_max(cls_prob, -1)
    cls = tf.argmax(cls_score, -1)
    cls = tf.cast(cls, tf.float32)
    cls = predictions[:, :, 4]
            
    boxes = _decode_box_predictions(anchor_boxes[None, ...], h_box)
    keypoints_xy = _decode_keypoint_predictions(anchor_boxes[None, ...], keypoints_xy)
    
    boxes_2d = tf.reshape(boxes, [-1, 4])    
    scores = tf.reshape(objectness, [-1, 1])
    
    cls = tf.reshape(cls, [-1, 1])
    
    keypoints_xy_3d = tf.reshape(keypoints_xy, [-1, NUM_KEYPOINT, 2])
    keypoints_cls_3d = tf.reshape(keypoints_cls, [-1, NUM_KEYPOINT, 1])    
    keypoints_3d = tf.concat((keypoints_xy_3d, keypoints_cls_3d), -1)
    keypoints_2d = tf.reshape(keypoints_3d, [-1, NUM_KEYPOINT * 3])
    
    ccbox = tf.concat((cls, scores, boxes_2d, keypoints_2d), -1)
    ccbox_check = tf.concat((boxes_2d, cls, keypoints_2d), -1)
    
    selected_indices, selected_scores = tf.image.non_max_suppression_with_scores(    
        ccbox[:, 2:2+4],
        ccbox[:, 1],        
        max_detections,
        nms_iou_threshold,
        confidence_threshold,        
    )
    output = tf.gather(ccbox_check, selected_indices)        
    return output   

In [ ]:
anchor_check = AnchorBox()
anchors = anchor_check.get_anchors_check(512,512)
anchor_sum = 0
for anchor in anchors:
    print(anchor.shape, anchor[-1], 'sqrt', np.sqrt(anchor.shape[0]/anchor_k))
    anchor_sum +=anchor.shape[0]
print('anchor_sum', anchor_sum)

In [ ]:
channel_label

In [ ]:
for image, multi_y in val_dataset:
    target = multi_y['detect']
    heatmap = multi_y['heatmap']
    print('image', image.shape)    
    print('target', target.shape)# (1, 12096, 56)    
    detections = decode_debug(image, target, confidence_threshold=0.15, nms_iou_threshold=0.2)
    print('detections', detections.shape)
    print(detections[0])
    target_cp = target[:, :, 4+1:]
    target_cp_xy = target_cp[:, :, :NUM_KEYPOINT*2]
    print('target_cp', target_cp.shape)
    #max tf.Tensor(187.73685, shape=(), dtype=float32) tf.Tensor(-219.20311, shape=(), dtype=float32)
    print('max', tf.reduce_max(target_cp_xy), tf.reduce_min(target_cp_xy))#
    break    

In [ ]:
print(list_y_train[0][0].shape)
print(list_y_train[0][0])

# 인코딩 라벨 어디감?
- 65번 가운데 여자(iou thresh 높이면 나옴)
- 73번 가운데 남자
- 76번 할아버지(label positive 낮추면 나옴)


In [ ]:
i = 0
for image, multi_y in val_dataset:
    i += 1
    target = multi_y['detect']
    heatmap = multi_y['heatmap']        
    detections = decode_debug(image, target, confidence_threshold=0.25, nms_iou_threshold=0.9)
    print(i, 'detections', detections.shape, 'target', target.shape)    
    kp = detections[:, -NUM_KEYPOINT * NUM_KEYPOINT_CH:]
    kp_cls = kp[:, 2::3]
    print(detections.shape) #양복 2명 할아버지 (7, 56)
    print(detections[:, :5])
    x = np.array(image[0], np.uint8)
    h = np.array(detections)
    
    img = Image.fromarray(x)        
    draw_box_keypoint(img, h, is_show_class=False)
    display(img)    
    


### model

In [ ]:
netbase = keras.applications.EfficientNetB3(include_top=False, input_shape=[512, 512, 3])
netbase.summary()


In [ ]:
def get_backbone():
    backbone = keras.applications.EfficientNetB3(include_top=False, input_shape=[None, None, 3])
    c2_output, c3_output, c4_output, c5_output = [
        backbone.get_layer(layer_name).output
        for layer_name in ["block2c_add", "block3c_add", "block5d_add", "top_activation"]]#block5c_add, block6d_add    
    return keras.Model(
        inputs=[backbone.inputs], outputs=[c2_output, c3_output, c4_output, c5_output]
    )

#D0 for layer_name in ["block2b_add", "block3b_add", "block5c_add", "block6d_add"]]
#B2 for layer_name in ["block2c_add", "block3c_add", "block5d_add", "top_activation"]]
#B3 for layer_name in ["block2c_add", "block3c_add", "block5e_add", "top_activation"]]
#D7 for layer_name in ["block2f_add", "block3g_add", "block5j_add", "block6d_add"]]

In [ ]:
def BifeaturePyramidNet(c345, filters):    
    a2 = c345[0]
    a3 = c345[1]
    a4 = c345[2]
    a5 = c345[3]
    
    regulizer  = tf.keras.regularizers.L2(l1)
    
    a2_0 = Conv2D(filters, 1, 1, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(a2)    
    a33 = Conv2D(filters*2, 1, 1, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(a3)
    a44 = Conv2D(filters*2, 1, 1, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(a4)
    a55 = Conv2D(filters*2, 1, 1, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(a5)
    a66 = Conv2D(filters*2, 3, 2, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(a5)
    
    a3_0, a3_1 = tf.split(a33, 2, -1)
    a4_0, a4_1 = tf.split(a44, 2, -1)
    a5_0, a5_1 = tf.split(a55, 2, -1)
    a6_0, a6_1 = tf.split(a66, 2, -1)
    
    a7_1 = Conv2D(filters, 3, 2, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(a6_0)     
    b7 = Conv2D(filters, 3, 2, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(a6_0)    
    b6 = a6_0
    
    a6_up = keras.layers.UpSampling2D(2)(a6_1)    
    b5 = keras.layers.Add()([a5_0, a6_up])  
        
    a5_up = keras.layers.UpSampling2D(2)(a5_1)    
    b4 = keras.layers.Add()([a4_0, a5_up])  
    
    b4_up = keras.layers.UpSampling2D(2)(b4)
    b3 = keras.layers.Add()([a3_0, b4_up])  
    
    b3_up = keras.layers.UpSampling2D(2)(b3)
    b2 = keras.layers.Add()([a2_0, b3_up])
    
    b2_down = Conv2D(filters, 3, 2, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(b2)
    b3_1 = Conv2D(filters, 1, 1, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(b3)    
    c3 = keras.layers.Add()([a3_1, b3_1, b2_down])
    
    c3_down = Conv2D(filters, 3, 2, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(c3)
    b4_1 = Conv2D(filters, 1, 1, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(b4)    
    c4 = keras.layers.Add()([a4_1, b4_1, c3_down])    
    
    c4_down = Conv2D(filters, 3, 2, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(c4)
    b5_1 = Conv2D(filters, 1, 1, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(b5)    
    c5 = keras.layers.Add()([a5_1, b5_1, c4_down])    
    
    c5_down = Conv2D(filters, 3, 2, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(c5)
    b6_1 = Conv2D(filters, 1, 1, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(b6)    
    c6 = keras.layers.Add()([a6_1, b6_1, c5_down])
    
    c6_down = Conv2D(filters, 3, 2, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(c6)
    b7_1 = Conv2D(filters, 1, 1, "same", groups=1, activation=activation, kernel_regularizer=regulizer)(b7)
    c7 = keras.layers.Add()([a7_1, b7_1, c6_down])
    return c3, c4, c5, c6, c7

In [ ]:
2**level_start

In [ ]:
def get_heatmap(nets, img_h, img_w):
    
    ch = NUM_KEYPOINT + 1
    heatmap_0 = nets[0][:, :, :, :ch]
    heatmap_1 = nets[1][:, :, :, :ch]
    heatmap_2 = nets[2][:, :, :, :ch]
    stride = 2 ** level_start        
    #heatmap_1 = shuffle_pixel_2x(heatmap_1, img_h//stride//2, img_w//stride//2, ch * 4)
    
    heatmap_fullsize_0 = tf.image.resize(heatmap_0, [img_h, img_w])
    heatmap_fullsize_1 = tf.image.resize(heatmap_1, [img_h, img_w])
    heatmap_fullsize_2 = tf.image.resize(heatmap_2, [img_h, img_w])
    heatmap = heatmap_fullsize_0 + heatmap_fullsize_1 + heatmap_fullsize_2
    return heatmap

In [ ]:
def createNet(num_classes, num_keypoints, num_keypoint_classes, anchor_k, is_train=False, is_backbone_train=True):    
    inputs = Input(shape=(None, None, 3))            
    prior_probability = tf.constant_initializer(-np.log((1 - 0.01) / 0.01))
    kernel_init = tf.initializers.he_normal()
    
    backbone = get_backbone()
    backbone.trainable = is_backbone_train
    nets_3 = backbone(inputs, training=is_train) 
    ch = 256
    p34567 = BifeaturePyramidNet(nets_3, ch)    
    
    cls_outputs = []
    box_outputs = []
    
    kernel_init = tf.initializers.he_normal()
    regulizer = tf.keras.regularizers.L2(l1)
    
    filters = 4 + 1 + num_classes + num_keypoints * (2 + num_keypoint_classes)
    print('filters', filters)
    
    conv_inter = keras.layers.Conv2D(ch, 3, activation=activation, padding="same", kernel_regularizer=regulizer, name='head_inter_0')
    conv_h0 = keras.layers.Conv2D(anchor_k * filters, 1, padding="same", name='head_0', kernel_initializer=kernel_init)   
    conv_h1 = keras.layers.Conv2D(anchor_k * filters, 3, padding="same", name='head_1', kernel_initializer=kernel_init)   
    conv_h2 = keras.layers.Conv2D(anchor_k * filters, 3, padding="same", name='head_2', kernel_initializer=kernel_init)
    conv_h3 = keras.layers.Conv2D(anchor_k * filters, 3, padding="same", name='head_3', kernel_initializer=kernel_init)
    conv_kernels = [0, conv_h0, conv_h1, conv_h2, conv_h3]
    conv_kernels = [0, conv_h0, conv_h0, conv_h0, conv_h0]
    
    drop = keras.layers.Dropout(0.001)
    N = tf.shape(nets_3[0])[0]
    
    cbox_outputs = []    
    
    for i in range(1, len(p34567)):            
        feature = drop(p34567[i])
        #feature = add_map(feature)
        conv_kernel = conv_kernels[i]
        cls_out = conv_kernel(conv_inter(feature))        
        cbox_out = tf.reshape(cls_out, [N, -1, filters])
        cbox_outputs.append(cbox_out[:,:,:])
    
    outputs = tf.concat(cbox_outputs, axis=1)
    img_h = tf.shape(inputs)[1]
    img_w = tf.shape(inputs)[2]
    heatmap = get_heatmap(p34567, img_h, img_w)    
    
    outputs = {'detect':outputs, 'heatmap':heatmap}        
    return keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
class BoxLoss(tf.losses.Loss):

    def __init__(self, delta):
        super(BoxLoss, self).__init__(reduction="none", name="BoxLoss")
        self._delta = delta

    def call(self, y_true, y_pred):    
        
        difference = y_true - y_pred
        absolute_difference = tf.abs(difference)        
        loss = tf.where(
            tf.less(absolute_difference, self._delta),
            0.5 * (difference ** 2),
            absolute_difference - 0.5,
        )
        loss = tf.where(loss < 0.05, 0.0, loss)#new marginal loss        
        return tf.reduce_sum(loss, axis=-1)

class ClassificationLoss(tf.losses.Loss):

    def __init__(self, alpha, gamma, num_classes):
        super(ClassificationLoss, self).__init__(reduction="none", name="ClassificationLoss")
        self._alpha = alpha
        self._gamma = gamma
        self._num_classes = num_classes
        
        
    def call(self, y_cls, y_pred):
        y_cls_int = tf.cast(y_cls, dtype=tf.int32)
        y_hot = tf.one_hot(y_cls_int, depth=self._num_classes, dtype=tf.float32,)
        
        y_positive = tf.cast(y_cls > 0, tf.float32)#finetune, 1:unknown
        y_positive_identity = tf.cast(y_cls > 1, tf.float32)# 1:unknown
        
        obj_score = tf.identity(y_pred[:, :], name='obj_score')
        #cls_score = y_pred[:, :, 1:1+self._num_classes]
        
        pt = tf.clip_by_value(obj_score, 1e-7, 1.0 - 1e-7)
                
        loss_p = - (1.0 - self._alpha) * tf.pow(1.0 - pt, self._gamma) * y_positive * tf.math.log(pt)        
        loss_f = - self._alpha * tf.pow(pt, self._gamma) * (1 - y_positive) * tf.math.log(1 - pt)
        loss_obj = loss_p + loss_f
          
        if False:
            cls_pt = tf.nn.softmax(cls_score)        
            cls_pt = tf.clip_by_value(cls_pt, 1e-7, 1.0 - 1e-7)
            loss_cls_p = - tf.pow(1.0 - cls_pt, self._gamma) * y_hot * tf.math.log(cls_pt)
            loss_cls_f = - tf.pow(cls_pt, self._gamma) * (1 - y_hot) * tf.math.log(1 - cls_pt)
            loss_cls = tf.reduce_sum(loss_cls_p + loss_cls_f, axis=-1)        
            is_various_cls_exist = tf.cast(tf.math.reduce_std(y_cls) > 0, tf.float32)                
        #loss_cls = is_various_cls_exist * y_positive * loss_cls
                        
        #normalizer = tf.reduce_sum(y_positive_identity, axis=-1)
        #loss_cls = tf.math.divide_no_nan(tf.reduce_sum(loss_cls, axis=-1), normalizer)                        
        
        loss = loss_obj# + .1*loss_cls#when not stable                
        return loss   

In [ ]:
class KeypointLoss(tf.losses.Loss):
    """Implements Smooth L1 loss"""

    def __init__(self, delta, gamma, num_classes):
        super(KeypointLoss, self).__init__(reduction="none", name="KeypointLoss")
        self._delta = delta
        self._gamma = gamma
        self._num_classes = num_classes
        
    def get_xy_loss(self, y_true, y_pred):        
        difference = y_true - y_pred
        absolute_difference = tf.abs(difference)        
        loss = tf.where(
            tf.less(absolute_difference, self._delta),
            0.5 * (difference ** 2),
            absolute_difference - 0.5,
        )
        #loss_xy = tf.where(loss < 0.05, 0.0, loss)#new marginal loss        
        loss = tf.reduce_mean(loss, axis=-1)
        return loss
    
    def get_cls_loss(self, y_cls, y_pred):        
        loss = tf.square(y_cls - y_pred)
        return loss    

    def call(self, y_true, y_pred):    
        #y: [m, anchors, NUM_KEYPOINT * (2 + classes)]        
        gt_x, gt_y, gt_cls = split_gt_keypoint(y_true)        
        h_x, h_y, h_cls, h_kp_cls = split_h_keypoint(y_pred)
        
        gt_xy = tf.stack((gt_x, gt_y), -1)
        h_xy = tf.stack((h_x, h_y), -1) 
        m = tf.shape(y_true)[0]
        #h_cls_score = tf.reshape(h_cls, [m, -1, NUM_KEYPOINT, NUM_KEYPOINT_CLASS])
        
        loss_cls = self.get_cls_loss(gt_cls, h_cls)#(m, anchor)
        loss_xy = self.get_xy_loss(gt_xy, h_xy)#(m, anchor, NUM_KEYPOINT)        
        
        is_visible = tf.cast(gt_cls > 0, tf.float32)        
        is_exist_kp_label = tf.cast(tf.reduce_any(gt_cls > 0, -1, True), tf.float32)
        
        kp_cls_sum_per_anchor = 1 + tf.reduce_sum(gt_cls, -1)
        weight_kp = kp_cls_sum_per_anchor / tf.reduce_max(kp_cls_sum_per_anchor, -1, True)
        
        loss_xy = gt_cls * loss_xy 
        loss_cls = loss_cls * is_exist_kp_label
        loss = 1 * tf.reduce_mean(loss_xy + loss_cls, axis=-1) #(m, anchor)         
        return loss

In [ ]:
NUM_KEYPOINT, NUM_KEYPOINT_CLASS

In [ ]:
class NetLoss(tf.losses.Loss):
    """Wrapper to combine both the losses"""

    def __init__(self, num_classes=80, num_keypoint_classes=3, alpha=0.3, gamma=3.0, delta=1.0):#alpha=0.25
        super(NetLoss, self).__init__(reduction="auto", name="NetLoss")
        self._clf_loss = ClassificationLoss(alpha, gamma, num_classes)
        self._box_loss = BoxLoss(delta)        
        self._keypoint_loss = KeypointLoss(delta, gamma, num_keypoint_classes)                

    def call(self, y_true, y_pred):
        
        y_box, y_cls, y_keypoint = split_gt(y_true)
        h_box, h_obj, h_cls, h_keypoint = split_hyperthesis(y_pred)
        gt_kp_x, gt_kp_y, gt_kp_cls = split_gt_keypoint(y_true)
        
        positive_mask = tf.greater(y_cls, 0.0)
        ignore_mask = tf.less(y_cls, 0.0)
        
        clf_loss = self._clf_loss(y_cls, h_obj)
        box_loss = self._box_loss(y_box, h_box) 
        keypoint_loss = self._keypoint_loss(y_keypoint, h_keypoint) 
        
        kp_cls_sum_per_anchor = 1 + tf.reduce_sum(gt_kp_cls, -1)
        weight_kp = kp_cls_sum_per_anchor / tf.reduce_max(kp_cls_sum_per_anchor, -1, True)
        
        clf_loss = tf.where(ignore_mask, 0.0, clf_loss)                
        box_loss = tf.where(positive_mask, box_loss, 0.0)
        keypoint_loss = tf.where(positive_mask, keypoint_loss, 0.0)
        loss = clf_loss + weight_kp * (box_loss + 0.1 * keypoint_loss)
        
        positive_mask = tf.cast(positive_mask, tf.float32)        
        normalizer = tf.reduce_sum(positive_mask, axis=-1)
        normalizer = tf.sqrt(normalizer)
        loss = tf.math.divide_no_nan(tf.reduce_sum(loss, axis=-1), normalizer)        
        return loss 

In [ ]:
def focal_loss(y_cls, score, class_k, gamma):
    y_cls_int = tf.cast(y_cls, dtype=tf.int32)
    y_hot = tf.one_hot(y_cls_int, depth=class_k, dtype=tf.float32)
        
    cls_pt = tf.nn.softmax(score)        
    cls_pt = tf.clip_by_value(cls_pt, 1e-7, 1.0 - 1e-7)
    loss_cls_p = - tf.pow(1.0 - cls_pt, gamma) * y_hot * tf.math.log(cls_pt)
    loss_cls_f = - tf.pow(cls_pt, gamma) * (1 - y_hot) * tf.math.log(1 - cls_pt)
    loss_cls = tf.reduce_sum(loss_cls_p + loss_cls_f, axis=-1) 
    return loss_cls

In [ ]:
class HeatmapLoss(tf.losses.Loss):
    """Wrapper to combine both the losses"""

    def __init__(self, num_classes, alpha=0.3, gamma=3.0, delta=1.0):#alpha=0.25
        super(HeatmapLoss, self).__init__(reduction="auto", name="HeatmapLoss")
        self._delta = delta
        self._gamma = gamma
        self._num_classes = num_classes

    def call(self, y_true, y_pred):
        #(m, h, w, m_classes)
        y_cls_int = tf.cast(y_true, dtype=tf.int32)                        
        #[1,32,32,18] vs. [1,32,32,7,18] mul
        loss = focal_loss(y_true, y_pred, self._num_classes, self._gamma)        
        #loss = loss * tf.cast(y_true > 0, tf.float32)
        loss = tf.boolean_mask(loss, y_true > 0)
        return loss 

In [ ]:
def recall(y_true, y_pred):    
    y_cls = tf.cast(y_true[:, :, 4], tf.int32)
    y_positive = tf.cast(y_cls > 0, tf.int32)
    y_bg = tf.cast(tf.abs(y_cls)==0, tf.int32)
    h_score = y_pred[:, :, 4]
    h_prob = tf.nn.sigmoid(h_score)    
    h_postive = tf.cast(tf.round(h_prob), tf.int32)
    
    true_positives = tf.cast(tf.logical_and(y_cls > 0, h_postive>0), tf.float32)
    false_negative = y_positive * (1 - h_postive)
                
    tp = tf.reduce_sum(true_positives, axis=1)# + 0.01
    fn = tf.reduce_sum(false_negative, axis=1)
    tp = tf.cast(tp, tf.float32)
    fn = tf.cast(fn, tf.float32)
    
    rec = tp / (tp + fn + 1e-8)
    return rec

def precision(y_true, y_pred):
    
    y_cls_symbol = tf.cast(y_true[:, :, 4], dtype=tf.int32)    
    y_cls_symbol = tf.cast(y_cls_symbol != 0, tf.int32)
    h_obj_prob = tf.nn.sigmoid(y_pred[:, :, 4])
    h_cls_symbol = tf.round(h_obj_prob)    
    h_cls_symbol = tf.cast(h_cls_symbol, tf.int32)
    
    true_positives = y_cls_symbol * h_cls_symbol
    false_positive = (1 - y_cls_symbol) * h_cls_symbol
    
    ones = tf.ones_like(true_positives)
    zeeros = tf.zeros_like(true_positives)
    true_positives = tf.cast(tf.equal(true_positives, ones), tf.float32)
    false_positive = tf.cast(tf.equal(false_positive, ones), tf.float32)
    
    tp = tf.reduce_sum(true_positives, axis=1)# + 0.01
    fp = tf.reduce_sum(false_positive, axis=1)
    tp = tf.cast(tp, tf.float32)
    fp = tf.cast(fp, tf.float32)
    prec = tp / (tp + fp + 1e-8)
    return prec

def acc(y_true, y_pred):    
    y_cls = tf.cast(y_true[:, :, 4], tf.int32)
    y_positive = y_cls > 0    
    h_cls = tf.math.argmax(y_pred[:, :, 5:5+num_classes], -1, output_type=tf.int32)        
    acc = tf.boolean_mask(tf.equal(y_cls, h_cls), y_positive)    
    #acc = tf.equal(y_cls, h_cls)
    return acc

In [ ]:
def kp_L1(y_true, y_pred):#keypoint
        
    y_box, y_cls, y_keypoint = split_gt(y_true)
    h_box, h_obj, h_cls, h_keypoint = split_hyperthesis(y_pred)   
    
    gt_x, gt_y, gt_cls = split_gt_keypoint(y_keypoint)
    h_x, h_y, h_cls, h_kp_cls = split_h_keypoint(h_keypoint)

    gt_xy = tf.stack((gt_x, gt_y), -1)
    h_xy = tf.stack((h_x, h_y), -1)     
    
    l1_distance = tf.reduce_mean(tf.abs(gt_xy - h_xy), [-1])#(38, 34)
    cond = tf.logical_and(tf.expand_dims(y_cls, -1) > 0, gt_cls > 0)
    l1_distance = tf.boolean_mask(l1_distance, cond)
    return l1_distance   

def kp_cls_acc(y_true, y_pred):#keypoint
    y_box, y_cls, y_keypoint = split_gt(y_true)    
    h_box, h_obj, h_cls, h_keypoint = split_hyperthesis(y_pred)
    
    gt_x, gt_y, gt_cls = split_gt_keypoint(y_keypoint)
    h_x, h_y, h_kp_cls_score, h_kp_cls = split_h_keypoint(h_keypoint)

    gt_xy = tf.concat((gt_x, gt_y), -1)
    h_xy = tf.concat((h_x, h_y), -1) 
    m = tf.shape(y_true)[0]
        
    #[1,14160,17] vs. [1,12240,17]    
    gt_cls = tf.cast(gt_cls, tf.int32)
    h_kp_cls = tf.cast(h_kp_cls, tf.int32)
    acc = tf.equal(gt_cls, h_kp_cls)
    #is_exist_kp_label = tf.cast(tf.reduce_any(gt_cls > 0, -1, True), tf.float32)
    cond = tf.logical_and(y_cls > 0, tf.reduce_any(gt_cls > 0, -1))
    acc = tf.boolean_mask(acc, cond)
    return acc   

In [ ]:
def heatmap_acc(y_true, y_pred):#keypoint
    
    h_cls = tf.argmax(y_pred, -1)    
    h_cls = tf.cast(h_cls, tf.int32)
    y_cls = tf.cast(y_true, tf.int32)
    
    acc = tf.equal(y_cls, h_cls) 
    acc = tf.boolean_mask(acc, y_true > 0)
    return acc

In [ ]:
np.set_printoptions(precision=3, linewidth=200)
image_height, image_width = padded_image_shape

img_check = 0
for image, multi_y in val_dataset:
    output_map = multi_y['detect']
    heatmap = multi_y['heatmap']
    print('output_map', output_map.shape)
    cbbox = output_map    
    bbox = cbbox[:, :, :4]
    cls_gt = cbbox[:,:,4]
    img_m, image_height, image_width, image_ch = image.shape
    anchor_feature_size = [(np.ceil(image_height / 2 ** i), np.ceil(image_width / 2 ** i)) 
                           for i in range(level_start, level_end)]
    print('anchor_feature_size', anchor_feature_size)    
    m = len(cbbox)    
    positive_count = np.sum(cls_gt>0)
    print('cbbox', cbbox.shape)
    print('cls_sum',np.sum(cls_gt < 0.0), np.sum(cls_gt == 0.0), 
          np.sum(cls_gt == 1.0), np.sum(cls_gt > 1.0))
    print('cls_mean',np.mean(cls_gt < 0.0), np.mean(cls_gt == 0.0), 
          np.mean(cls_gt == 1.0), np.mean(cls_gt > 0.0))
    print('shape',image.shape, cbbox.shape,'unique', np.unique(cls_gt))
    print('anchor_feature_size', anchor_feature_size)
    offset = 0
    positive_maps = []
    for anchor_feature_size_1 in anchor_feature_size:        
        fm_h, fm_w = anchor_feature_size_1
        fm_h = int(fm_h)
        fm_w = int(fm_w)        
        fm_wh = int(fm_h * fm_w * anchor_k)
        cbbox_anchor = cbbox[:, offset:offset+fm_wh, 4]
        cbbox_anchor = np.reshape(cbbox_anchor, [m, fm_h, fm_w, anchor_k])
        coount_m1 = np.count_nonzero(cbbox_anchor==-1)
        coount_0 = np.count_nonzero(cbbox_anchor==0)
        coount_1 = np.count_nonzero(cbbox_anchor==1)
        coount_1_over = np.count_nonzero(cbbox_anchor>1)
        positive_ratio = np.mean(cbbox_anchor>0)
        positive_maps.append(cbbox_anchor>0)
        print('cbbox_anchor', cbbox_anchor.shape, coount_m1, coount_0, coount_1, coount_1_over, 'ratio', positive_ratio)
        sample_0_cbbox = cbbox_anchor[0]
        sample_0_cbbox_sum = np.max(sample_0_cbbox, -1).astype(np.int)       
      
        offset += fm_wh
        if False:            
            file_name = str(fm_h)+ '_' + str(fm_w)+ '.txt'
            np.savetxt(file_name,sample_0_cbbox_sum, fmt='%d',delimiter='')
    img_check = image
    break

In [ ]:
pmap0 = np.array(Image.fromarray(np.max(positive_maps[0][0],-1)).resize((image_width, image_height)))
pmap1 = np.array(Image.fromarray(np.max(positive_maps[1][0],-1)).resize((image_width, image_height)))
pmap2 = np.array(Image.fromarray(np.max(positive_maps[2][0],-1)).resize((image_width, image_height)))
#pmap3 = np.array(Image.fromarray(np.max(positive_maps[3][0],-1)).resize((image_width, image_height)))
#pmap4 = np.array(Image.fromarray(np.max(positive_maps[4][0],-1)).resize((image_width, image_height)))
pmap0 = pmap0.astype(np.uint8)
pmap1 = pmap1.astype(np.uint8)
pmap2 = pmap2.astype(np.uint8)
pmap3 = 0#pmap3.astype(np.uint8)
pmap4 = 0#pmap4.astype(np.uint8)

In [ ]:
pmap_with_img = np.array(img_check)[0]#*255
pmap_with_img = pmap_with_img.astype(np.uint8)
pmap_add = np.expand_dims(pmap0+pmap1+pmap2+pmap3+pmap4, -1)
pmap = pmap_add*(255//np.max(pmap_add))
mix_rgb = np.concatenate((pmap, pmap_with_img[:,:,1:]),-1)
plt.figure(figsize=(10,10))
plt.imshow(mix_rgb)
plt.title(str(np.mean(pmap_add)))

In [ ]:
def stack_image(img0, img1):
    a = np.array(img0)
    b = np.array(img1)
    return Image.fromarray(np.concatenate((a, b), axis=1))

def stack_image_3(img0, img1, img2):
    a = np.array(img0)
    b = np.array(img1)
    c = np.array(img2)
    return Image.fromarray(np.concatenate((a, b, c), axis=1))

In [ ]:
def compare_gt_h(check_dataset, stride=1):
    i = 0
    for image, target_dict in check_dataset:        
        i += 1
        if stride > 1 and i%stride!=0:
            continue

        x = np.array(image[0], np.uint8)
        y_detect = target_dict['detect']
        y_heatmap = target_dict['heatmap']
        detections_gt = decode_debug(image, y_detect, confidence_threshold=0.15, nms_iou_threshold=0.2)    
        model_out = inference_model.predict(image)  
        detections = model_out['detect']
        heatmap = model_out['heatmap']
        print('x', x.shape, 'detections', detections.shape)
        #print(detections)
        h = np.array(detections)  
        gt = np.array(detections_gt)
        print(i, 'gt', gt.shape, 'h', h.shape)
        
        heatmap_scale = (255//(1+NUM_KEYPOINT))
        heatmap_arr = np.array(heatmap[0]).astype(np.uint8) * heatmap_scale
        heatmap_arr = np.stack((heatmap_arr, 255-heatmap_arr, 255-heatmap_arr), -1)
        img_heatmap = Image.fromarray(heatmap_arr)
        
        img = Image.fromarray(x)
        img_copy = x.copy()
        img_copy[:, :, 0] = tf.cast(np.array(y_heatmap[0]*heatmap_scale), tf.uint8)
        
        img_gt = Image.fromarray(img_copy)
        draw_box_keypoint(img, h, is_show_class=False)
        draw_box_keypoint(img_gt, gt, is_show_class=False)
        draw_box_keypoint(img_heatmap, gt, is_show_class=False)        
        
        print('heatmap_arr', heatmap_arr.shape)
        display(stack_image_3(img, img_gt, img_heatmap))    
        #display(stack_image(img, img_gt))    

In [ ]:
def load_weight():           
    print('latest_checkpoint', path_weight)
    model.load_weights(path_weight)

In [ ]:
num_classes, NUM_KEYPOINT_CLASS

In [ ]:
#filters
4+1+num_classes+NUM_KEYPOINT*(2+NUM_KEYPOINT_CLASS)

In [ ]:
with strategy.scope():
    optimizer = tf.optimizers.SGD(learning_rate=1e-2, momentum=0.1)
    loss_detect = NetLoss(num_classes, NUM_KEYPOINT_CLASS)
    loss_heatmap = HeatmapLoss(NUM_KEYPOINT + 1)
    model = createNet(num_classes, NUM_KEYPOINT, 1, anchor_k, is_backbone_train=True)
    metric_det = [recall, precision, kp_L1, kp_cls_acc]

    losses = {"detect": loss_detect, 'heatmap': loss_heatmap}
    metrics = {"detect": metric_det, 'heatmap': heatmap_acc} 
    loss_weights = {"detect": 1, 'heatmap': 50} 

    model.compile(loss=losses, optimizer=optimizer, metrics=metrics, loss_weights=loss_weights)
    callbacks_list = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath=path_weight,
            monitor="loss",
            save_best_only=False,
            save_weights_only=True,
            verbose=0,
            save_freq=200
        )
    ]

In [ ]:
model.summary()#b1:13,906,006, b2:15,431,112, b3+:25,731,444

In [ ]:
print(len(list_x_train), len(list_x_test), batch_size)

In [ ]:
with strategy.scope():
    load_weight()

In [ ]:
out = model.evaluate(val_dataset.take(10))

In [ ]:
epochs = 100000
hist = model.fit(train_dataset, epochs=epochs, callbacks=callbacks_list, verbose=1)
'''
2s 177ms/step - loss: 0.5565 - tf_op_layer_concat_283_loss: 0.4345 - tf_op_layer_ResizeBilinear_16_loss: 0.0122 - tf_op_layer_concat_283_recall: 1.0000 - tf_op_layer_concat_283_precision: 1.0000 - tf_op_layer_concat_283_kp_L1: 0.7677 - tf_op_layer_concat_283_kp_cls_acc: 0.5468 - tf_op_layer_ResizeBilinear_16_hitmap_acc: 0.9748
'''

In [ ]:
model.save_weights(path_weight)
path_weight

In [ ]:
image = tf.keras.Input(shape=[None, None, 3], name="image")
predictions = model(image, training=False)
h_detect = predictions['detect']
h_heatmap_score = predictions['heatmap']
h_heatmap = tf.argmax(h_heatmap_score, -1)
detections = decodePredictions(image, h_detect, confidence_threshold=0.5, nms_iou_threshold=0.15)
model_out = {'detect':detections, 'heatmap':h_heatmap}
inference_model = tf.keras.Model(inputs=image, outputs=model_out)

In [ ]:
compare_gt_h(val_dataset, stride=1)

In [ ]:
compare_gt_h(val_dataset, stride=1)